## Import Libraries

In [71]:
# data processing
import pandas as pd

#linear algebra
import numpy as np

# data visualization
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

#regex
import re


#Algorithms

## Getting the Data

In [72]:
#online_coupon train data
df_on=pd.read_csv("ccf_online_stage1_train.csv")

#offline_coupon train data
df_off=pd.read_csv("ccf_offline_stage1_train.csv")

#offline_coupon test data
df_off_test=pd.read_csv("ccf_offline_stage1_test_revised.csv")

## Feature Engineering

## User level features

In [132]:
#creating training dataset to fed to the model

x = 'Null'
df_off_unique = (df_off.fillna(x).groupby(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date']).size().reset_index().rename(columns={0 : 'Count'}).replace(x,np.NaN))
df_off_unique.drop(['Count'],axis=1,inplace=True)


x = 'Null'    
df_on_unique = (df_on.fillna(x).groupby(['User_id', 'Merchant_id', 'Action', 'Coupon_id', 'Discount_rate',
                'Date_received', 'Date']).size().reset_index().rename(columns={0 : 'Count'}).replace(x,np.NaN))
df_on_unique.drop(['Count'],axis=1,inplace=True)






,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Redeem_date,Original_Price,Discounted_price,Rate,Target
0,4,1433,8735.0,30:5,10.0,2016-02-14,NaT,-1.0,30.0,5.0,0.833,0
1,4,1469,2902.0,0.95,10.0,2016-06-07,NaT,-1.0,100.0,5.0,0.950,0
2,35,3381,1807.0,300:30,0.0,2016-01-30,NaT,-1.0,300.0,30.0,0.900,0
3,35,3381,9776.0,10:5,0.0,2016-01-29,NaT,-1.0,10.0,5.0,0.500,0
4,35,3381,11951.0,200:20,0.0,2016-01-29,NaT,-1.0,200.0,20.0,0.900,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1716984,7360967,3103,2375.0,30:1,2.0,2016-01-20,NaT,-1.0,30.0,1.0,0.967,0
1716987,7360967,6485,10323.0,20:5,1.0,2016-03-22,NaT,-1.0,20.0,5.0,0.750,0
1716988,7361024,1433,8735.0,30:5,10.0,2016-02-11,NaT,-1.0,30.0,5.0,0.833,0
1716989,7361032,2146,11173.0,100:10,2.0,2016-01-29,NaT,-1.0,100.0,10.0,0.900,0


In [148]:
df_off_unique2=df_off_unique[df_off_unique['Date_received'].isna()==False].copy(deep=True)
df_off_unique2['Date'] = pd.to_datetime(df_off_unique2['Date'],format="%Y%m%d")
df_off_unique2['Date_received'] = pd.to_datetime(df_off_unique2['Date_received'],format="%Y%m%d")

df_off_unique2["Redeem_date"]=df_off_unique2["Date"].dt.date-df_off_unique2["Date_received"].dt.date
df_off_unique2["Redeem_date"]=df_off_unique2["Date"].dt.date-df_off_unique2["Date_received"].dt.date
df_off_unique2['Redeem_date'] = df_off_unique2['Redeem_date'].dt.days.astype('str')
df_off_unique2['Redeem_date'] = pd.to_numeric(df_off_unique2['Redeem_date'],errors="coerce")
split_data = df_off_unique2["Discount_rate"].str.split(":")
data = split_data.to_list()
for i in range(len(data)):
    if len(data[i])==1:
        data[i].insert(1,100-float(data[i][0])*100)
        data[i][0]=100
        

df_off_unique2['temp']=data
df_off_unique2[['Original_Price','Discounted_price']] = pd.DataFrame(df_off_unique2.temp.values.tolist(), index= df_off_unique2.index)
df_off_unique2["Original_Price"]=df_off_unique2["Original_Price"].astype(float)
df_off_unique2["Discounted_price"]=df_off_unique2["Discounted_price"].astype(float)
df_off_unique2["Rate"]=round((df_off_unique2["Original_Price"]-df_off_unique2["Discounted_price"])/df_off_unique2["Original_Price"],3)
del df_off_unique2["temp"] 
df_off_unique2["Redeem_date"].fillna(-1, inplace = True) 
df_off_unique2["Distance"].fillna(df_off_unique2["Distance"].mean(), inplace = True) 
df_off_unique2["Target"]=0
df_off_unique2.loc[(df_off_unique2["Redeem_date"]<=15) & (df_off_unique2["Redeem_date"]>=0), 'Target'] = 1 
df_off_unique2

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Redeem_date,Original_Price,Discounted_price,Rate,Target
0,4,1433,8735.0,30:5,10.0,2016-02-14,NaT,-1.0,30.0,5.0,0.833,0
1,4,1469,2902.0,0.95,10.0,2016-06-07,NaT,-1.0,100.0,5.0,0.950,0
2,35,3381,1807.0,300:30,0.0,2016-01-30,NaT,-1.0,300.0,30.0,0.900,0
3,35,3381,9776.0,10:5,0.0,2016-01-29,NaT,-1.0,10.0,5.0,0.500,0
4,35,3381,11951.0,200:20,0.0,2016-01-29,NaT,-1.0,200.0,20.0,0.900,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1716984,7360967,3103,2375.0,30:1,2.0,2016-01-20,NaT,-1.0,30.0,1.0,0.967,0
1716987,7360967,6485,10323.0,20:5,1.0,2016-03-22,NaT,-1.0,20.0,5.0,0.750,0
1716988,7361024,1433,8735.0,30:5,10.0,2016-02-11,NaT,-1.0,30.0,5.0,0.833,0
1716989,7361032,2146,11173.0,100:10,2.0,2016-01-29,NaT,-1.0,100.0,10.0,0.900,0


In [149]:
# no of coupons users received, redeemed and redemption rate
def create_user_crr(df):
    df2=df[df["Coupon_id"].isna()==False]
    df3=df2.groupby('User_id').agg({"Coupon_id":"count","Date":"count"})
    df3.rename(columns = {"Coupon_id": "Coupon_receive_count","Date":"Coupon_redeem_count"}, inplace = True) 
    df3["User_Redeem_Receive_Ratio"]=df3["Coupon_redeem_count"]/df3["Coupon_receive_count"]
    df3.reset_index(inplace=True)
    df3.sort_values(by=["Coupon_receive_count"],ascending=False,inplace=True)
    df=pd.merge(df,df3,on='User_id')
    return df

#avg distance of user from store
def create_avg_dist_user(df):
    df2=df_off_unique2.groupby('User_id').agg({"Distance":"mean"})
    df2.rename(columns = {"Distance": "User_avg_distance"}, inplace = True) 
    df2.reset_index(inplace=True)
    df=pd.merge(df,df2,on='User_id')
    return df

# no of different actions by users
def create_no_of_actions(df_off_unique2,df_on_unique,df_off_unique):
    list_on_off = set(df_on_unique["User_id"].unique()).intersection(set(df_off_unique["User_id"].unique()))
    df_on_off = df_on[df_on['User_id'].isin(list(list_on_off))]
    df=pd.get_dummies(df_on_off["Action"])
    df_on_off=pd.concat([df_on_off,df],axis=1)
    df_on_off.rename(columns = {0: "No_of_Clicks",1:"No_of_Buy",2:"No_of_Get_Coupons"}, inplace = True) 
    df2=df_on_off.groupby('User_id').agg({"No_of_Clicks":"count","No_of_Buy":"count","No_of_Get_Coupons":"count"})
    #df2["Buy_Click_Ratio"]=df2["No_of_Buy"]/df2["No_of_Clicks"]
    df2.reset_index(inplace=True)
    df_off_unique2=pd.merge(df_off_unique2,df2,on='User_id')
    return df_off_unique2

#active duration of users
def create_user_active_duration(df_off_unique2):
    df_temp=df_off_unique2.copy(deep=True)
    df_temp["DateOrder"]=df_temp["Date"]
    df_temp["DateOrder"].fillna(df_temp["Date_received"],inplace=True)
    df=df_temp.groupby("User_id").agg({"DateOrder": pd.Series.nunique})
    df.reset_index(inplace=True)
    df.columns=['User_id','ActiveDuration']
    #df2=df_temp.groupby("User_id").agg({"DateOrder": ['min',np.max],"Target":'max'})
    #df2.reset_index(inplace=True)
    #df2["ActiveDuration"]=df2.iloc[:,2]-df2.iloc[:,1]
    #df2.columns = ['User_id', 'min', 'amax', 'max', 'ActiveDuration'] 
    #df2['ActiveDuration'] = df2['ActiveDuration'].dt.days.astype('str')
    #df2['ActiveDuration'] = pd.to_numeric(df2['ActiveDuration'],errors="coerce")
    #del df2['min']
    #del df2['amax']
    #del df2['max']
    df_off_unique2=pd.merge(df_off_unique2,df,on="User_id")
    return df_off_unique2


def create_no_of_buys(df_off_unique2):
    df=df_off_unique2.groupby("User_id").agg({'Date':'count'})
    df.reset_index(inplace=True)
    df.columns=["User_id","No_of_buys"]
    df_off_unique2=pd.merge(df_off_unique2,df,on='User_id')
    return df_off_unique2

#merchant level crr
def create_merchant_crr(df):
    df2=df[df["Coupon_id"].isna()==False]
    df3=df2.groupby('Merchant_id').agg({"Coupon_id":"count","Date":"count"})
    df3.rename(columns = {"Coupon_id": "Coupon_sent_count","Date":"Coupon_redeem_count"}, inplace = True) 
    df3["Merchant_Redeem_Sent_Ratio"]=df3["Coupon_redeem_count"]/df3["Coupon_sent_count"]
    df3.reset_index(inplace=True)
    df3.sort_values(by=["Coupon_sent_count"],ascending=False,inplace=True)
    df=pd.merge(df,df3,on='Merchant_id')
    return df


def create_user_merchant_crr(df):
    df3=df.groupby(['User_id','Merchant_id']).agg({"Coupon_id":"count","Date":"count"})
    df3.rename(columns = {"Coupon_id": "U_M_sent","Date":"U_M_redeem"}, inplace = True) 
    df3["U_M_crr"]=df3["U_M_redeem"]/df3["U_M_sent"]
    df3.reset_index(inplace=True)
    df3.sort_values(by=["U_M_sent"],ascending=False,inplace=True)
    df=pd.merge(df_off_unique2,df3,on=['User_id','Merchant_id'],how='outer')
    return df


def create_user_merchant_visits(df):
    df3=df.groupby(['User_id','Merchant_id']).agg({"Date":pd.Series.nunique})
    df3.rename(columns = {"Date": "U_M_No_of_visits"}, inplace = True) 
    df3.reset_index(inplace=True)
    df=pd.merge(df,df3,on=['User_id','Merchant_id'],how='outer')
    return df

def create_user_merchant_distance(df):
    df3=df.groupby(['User_id','Merchant_id']).agg({"Distance":'mean'})
    df3.rename(columns = {"Distance": "U_M_Dist"}, inplace = True) 
    df3.reset_index(inplace=True)
    df=pd.merge(df,df3,on=['User_id','Merchant_id'],how='outer')
    return df

def create_merchant_no_of_buys(df_off_unique2):
    df=df_off_unique2.groupby("Merchant_id").agg({'Date':'count'})
    df.reset_index(inplace=True)
    df.columns=["Merchant_id","Merchant_No_of_buys"]
    df_off_unique2=pd.merge(df_off_unique2,df,on='Merchant_id')
    return df_off_unique2

def create_avg_dist_merchant(df):
    df2=df_off_unique2.groupby('Merchant_id').agg({"Distance":"mean"})
    df2.rename(columns = {"Distance": "Merchant_avg_distance"}, inplace = True) 
    df2.reset_index(inplace=True)
    df=pd.merge(df,df2,on='Merchant_id')
    return df

def create_merchant_active_duration(df_off_unique2):
    df_temp=df_off_unique2.copy(deep=True)
    df_temp["DateOrder"]=df_temp["Date"]
    df_temp["DateOrder"].fillna(df_temp["Date_received"],inplace=True)
    df2=df_temp.groupby("Merchant_id").agg({"DateOrder": ['min',np.max],"Target":'max'})
    df2.reset_index(inplace=True)
    df2["ActiveDuration"]=df2.iloc[:,2]-df2.iloc[:,1]
    df2.columns = ['Merchant_id', 'min', 'amax', 'max', 'ActiveDuration'] 
    df2['ActiveDuration'] = df2['ActiveDuration'].dt.days.astype('str')
    df2['ActiveDuration'] = pd.to_numeric(df2['ActiveDuration'],errors="coerce")
    del df2['min']
    del df2['amax']
    del df2['max']
    df_off_unique2=pd.merge(df_off_unique2,df2,on="Merchant_id")
    return df_off_unique2

In [150]:
df_off_unique2=create_user_crr(df_off_unique2)
df_off_unique2=create_avg_dist_user(df_off_unique2)
#df_off_unique2=create_no_of_actions(df_off_unique2,df_on_unique,df_off_unique)
df_off_unique2=create_user_active_duration(df_off_unique2)
df_off_unique2=create_no_of_buys(df_off_unique2)
df_off_unique2=create_merchant_crr(df_off_unique2)
df_off_unique2=create_user_merchant_crr(df_off_unique2)
#df_off_unique2=create_user_merchant_visits(df_off_unique2)
#df_off_unique2=create_user_merchant_distance(df_off_unique2)
df_off_unique2=create_merchant_no_of_buys(df_off_unique2)
df_off_unique2=create_avg_dist_merchant(df_off_unique2)
df_off_unique2=create_merchant_active_duration(df_off_unique2)



df_off_unique2

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Redeem_date,Original_Price,Discounted_price,...,No_of_buys,Coupon_sent_count,Coupon_redeem_count_y,Merchant_Redeem_Sent_Ratio,U_M_sent,U_M_redeem,U_M_crr,Merchant_No_of_buys,Merchant_avg_distance,ActiveDuration_y
0,4,1433,8735.0,30:5,10.000000,2016-02-14,NaT,-1.0,30.0,5.0,...,0,19900,726,0.036482,1,0,0.0,726,5.187776,164
1,236,1433,8735.0,30:5,0.000000,2016-02-11,NaT,-1.0,30.0,5.0,...,0,19900,726,0.036482,1,0,0.0,726,5.187776,164
2,318,1433,8735.0,30:5,6.000000,2016-02-14,NaT,-1.0,30.0,5.0,...,0,19900,726,0.036482,1,0,0.0,726,5.187776,164
3,327,1433,8735.0,30:5,3.000000,2016-02-12,NaT,-1.0,30.0,5.0,...,0,19900,726,0.036482,1,0,0.0,726,5.187776,164
4,2384,1433,9476.0,20:5,10.000000,2016-01-25,NaT,-1.0,20.0,5.0,...,0,19900,726,0.036482,1,0,0.0,726,5.187776,164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015384,7335410,1477,11724.0,30:1,3.281054,2016-05-16,2016-05-26,10.0,30.0,1.0,...,2,4,2,0.500000,4,2,0.5,2,3.281054,7
1015385,7335410,1477,11724.0,30:1,3.281054,2016-05-22,NaT,-1.0,30.0,1.0,...,2,4,2,0.500000,4,2,0.5,2,3.281054,7
1015386,7335410,1477,11724.0,30:1,3.281054,2016-05-29,NaT,-1.0,30.0,1.0,...,2,4,2,0.500000,4,2,0.5,2,3.281054,7
1015387,7337925,2779,8316.0,50:5,10.000000,2016-05-16,NaT,-1.0,50.0,5.0,...,1,2,1,0.500000,2,1,0.5,1,10.000000,21


In [151]:
df_off_unique2.to_csv("Train_data.csv")